In [1]:
import pandas as pd
import sqlite3

pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
#pd.set_option('max_colwidth', 5000)

db = 'mlb.db'

def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q, conn)

def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)
        
def show_tables():
    q = '''
    SELECT
        
        name,
        type,
        sql
    FROM sqlite_master
    WHERE type IN ("table","view");   
    '''
    return run_query(q)


In [2]:
log = pd.read_csv("game_log.csv",low_memory=False)

In [ ]:
print(log.shape)
display(log.head())
display(log.tail())
log.dtypes


Looks like games are chronologically ordered from years 1871 to 2016. The recent games have much higher attendance than earlier games decades ago. Their lengths have increased too. There were only data on hp umpires in the past but now there's 1b,2b,3b too. Could mean missing data, or there wasn't such a role then. The values in columns ending with player_n_def_pos may not always be n, meaning their batting order and defensive positions may be different. 

In [ ]:
log['forefeit'][log.forefeit.notnull()]

In [ ]:
park = pd.read_csv("park_codes.csv")
print(park.shape)
display(park.head())
display(park.tail())
park.describe(include='all')


In [ ]:
person = pd.read_csv("person_codes.csv")
print(person.shape)
display(person.head())
display(person.tail())
person.describe(include='all')

Person_codes gives more information about a person described as a player, manager, coach, or umpire with name or id in the main log. The same person could take more than 1 role at different times, and most start as a player within the 4 roles. Coaches don't seem to be recorded in the main log.

In [ ]:
team = pd.read_csv("team_codes.csv")
print(team.shape)
display(team.head())
display(team.tail())
team.describe(include='all')
display(team["franch_id"].value_counts().head())
team[team["end"] == 0]

Team_codes shows years in which teams were active. 2-letter League codes match those in main log too. 
It appears that teams move between leagues and cities. The team_id changes when this happens, franch_id (which is probably 'Franchise') helps us tie all of this together.

In [ ]:
logset = set(log.columns)
parkset = set(park.columns)
personset = set(person.columns)
teamset = set(team.columns)

display(logset & parkset)
display(logset & personset)
display(logset & teamset)


In [ ]:
log["h_league"].value_counts()
def league_info(league):
    league_games = log[log["h_league"] == league]
    earliest = league_games["date"].min()
    latest = league_games["date"].max()
    print("{} went from {} to {}".format(league,earliest,latest))


for league in log["h_league"].unique():
    league_info(league)

4 of the less famous leagues have been discontinued by 1915, the two left are National League and American League

In [ ]:
tables = {
    "game_log": log,
    "person_codes": person,
    "team_codes": team,
    "park_codes": park
}

with sqlite3.connect(db) as conn:
    for name,dataframe in tables.items():
        conn.execute("DROP TABLE IF EXISTS {};".format(name))
        dataframe.to_sql(name,conn,index=False)
        

In [ ]:
c1 = '''
    ALTER TABLE game_log
    ADD COLUMN game_id TEXT;
'''

try:
    run_command(c1)
except:
    pass

c2 = '''
 UPDATE game_log
    SET game_id =  date || h_name || number_of_game
 WHERE game_id IS NULL; 
'''
run_command(c2)

q = '''
    SELECT game_id,h_name, date, number_of_game 
    FROM game_log 
    LIMIT 5
'''
run_query(q)

# Looking for Normalization Opportunities

In park_codes, start and end represent first and last dates where games are played at that park, and league column can be found in game_log.
In person_codes, names are in game_log, debut dates can be calculated from main log by searching for smallest date recorded, assuming the game_log data is complete(began documenting before a person's debut)
In team_codes, start and end are the first and last games of each team, this can be inferred from game_log and the league that the team plays can be read directly from game_log. 
In game_log all columns beginning with h and v can be condensed by having 2 rows for each game.
Umpire columns can be condensed by adding the column summarizing the umpire type shown in the prefix of umpire columns
All names can be removed from game_log, leaving only id. Defensive position numbers can be combined.
Awards may be broken out.


![can't see](capture.jpg)


![](C:/Users/hanqi/Desktop/datasets/Capture.jpg)



# Adding Tables without Foreign Keys: person, park, league, appearance_type

In [ ]:
c1 = 'DROP TABLE IF EXISTS person;'
    
c2 = '''
    CREATE TABLE person
    (
    person_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
    )  
'''
c3 = '''
    INSERT INTO person
    SELECT 
        id, 
        first, 
        last 
    FROM person_codes
'''


run_command(c1)
run_command(c2)
run_command(c3)

q = "select * from person LIMIT 5"
run_query(q)

In [ ]:

    
c1 = '''
    CREATE TABLE IF NOT EXISTS park
    (
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
    )  
'''
c2 = '''
    INSERT OR IGNORE INTO park
    SELECT 
        park_id, 
        name, 
        aka, 
        city, 
        state, 
        notes 
    FROM park_codes
'''


run_command(c1)
run_command(c2)

q = "select * from park LIMIT 5"
run_query(q)

In [ ]:

    
c1 = '''
    CREATE TABLE IF NOT EXISTS league
    (
    league_id TEXT PRIMARY KEY,
    league_name TEXT
    )  
'''
c2 = '''
    INSERT OR IGNORE INTO league (
        league_id,
        league_name)
    VALUES 
    ('NL','National League'),
    ('AL','American League'),
    ('AA','American Association'),
    ('FL','Federal League'),
    ('PL','Players League'),
    ('UA','Union Association')
    
'''

run_command(c1)
run_command(c2)


q = "select * from league LIMIT 5"
run_query(q)

# Using Pandas library

In [ ]:
c1 = """
CREATE TABLE IF NOT EXISTS appearance_type (
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT
);
"""
run_command(c1)

appearance_type = pd.read_csv('appearance_type.csv')

with sqlite3.connect('mlb.db') as conn:
    appearance_type.to_sql('appearance_type',
                           conn,
                           index=False,
                           if_exists='append')

q = """
SELECT * FROM appearance_type;
"""

run_query(q)

# Using the Python SQLite library

In [ ]:
import csv

with open("appearance_type.csv") as f:
    data = list(csv.reader(f))[1:]

    
c1 = "DROP TABLE IF EXISTS appearance_type;"

run_command(c1)

c2 = """
CREATE TABLE appearance_type (
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT
);
"""
c3 = '''
    INSERT INTO appearance_type(
    appearance_type_id,
    name,
    category
    )
    VALUES 
    (?,?,?)
'''


run_command(c1)
run_command(c2)

with sqlite3.connect(db) as conn:
    cur = conn.cursor()
    cur.executemany(c3, data)

q = """
SELECT * FROM appearance_type;
"""

run_query(q)

# Adding tables with Foreign Keys: team, game

In [ ]:
c1 = """
CREATE TABLE IF NOT EXISTS team (
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);
"""

c2 = """
INSERT OR IGNORE INTO team
SELECT
    team_id,
    league,
    city,
    nickname,
    franch_id
FROM team_codes;
"""

q = """
SELECT * FROM team
LIMIT 5;
"""

run_command(c1)
run_command(c2)
run_query(q)

In [ ]:

c1 = """
CREATE TABLE IF NOT EXISTS game (
    game_id TEXT PRIMARY KEY,
    date TEXT,
    number_of_game INTEGER,
    park_id TEXT,
    length_outs INTEGER,
    day BOOLEAN,
    completion TEXT,
    forefeit TEXT,
    protest TEXT,
    attendance INTEGER,
    legnth_minutes INTEGER,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES park(park_id)
);
"""

c2 = """
INSERT OR IGNORE INTO game
SELECT
    game_id,
    date,
    number_of_game,
    park_id,
    length_outs,
    CASE
        WHEN day_night = "D" THEN 1
        WHEN day_night = "N" THEN 0
        ELSE NULL
        END
        AS day,
    completion,
    forefeit,
    protest,
    attendance,
    length_minutes,
    additional_info,
    acquisition_info
FROM game_log;
"""

q = """
SELECT * FROM game
LIMIT 5;
"""

run_command(c1)
run_command(c2)
run_query(q)

In [ ]:
c1 = """
CREATE TABLE IF NOT EXISTS team_appearance (
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id)
);
"""

run_command(c1)

c2 = """
INSERT OR IGNORE INTO team_appearance
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    FROM game_log

UNION

    SELECT    
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    from game_log;
"""

run_command(c2)

q = """
SELECT * FROM team_appearance
ORDER BY game_id DESC 
LIMIT 5
"""

run_query(q)

In [54]:
c0 = "DROP TABLE IF EXISTS person_appearance"

run_command(c0)

c1 = """
CREATE TABLE person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);
"""

c2 = """
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL    

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE "1b_umpire_id" IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL;
"""

template = """
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""

run_command(c1)
run_command(c2)

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        run_command(template.format(**query_vars))

In [55]:
print(run_query("SELECT COUNT(DISTINCT game_id) games_game FROM game"))
print(run_query("SELECT COUNT(DISTINCT game_id) games_person_appearance FROM person_appearance"))

q = """
SELECT
    pa.*,
    at.name,
    at.category
FROM person_appearance pa
INNER JOIN appearance_type at on at.appearance_type_id = pa.appearance_type_id
WHERE PA.game_id = (
                   SELECT max(game_id)
                    FROM person_appearance
                   )
ORDER BY team_id, appearance_type_id
"""

run_query(q)

   games_game
0      171907
   games_person_appearance
0                   171907


,appearance_id,person_id,team_id,game_id,appearance_type_id,name,category
0,1646109,porta901,None,20161002WAS0,U1B,First Base,umpire
1,1646108,onorb901,None,20161002WAS0,U2B,Second Base,umpire
2,1646107,kellj901,None,20161002WAS0,U3B,Third Base,umpire
3,1646110,tumpj901,None,20161002WAS0,UHP,Home Plate,umpire
4,1646111,brica001,MIA,20161002WAS0,ALP,Losing Pitcher,award
5,6716279,koeht001,MIA,20161002WAS0,D1,Pitcher,defense
6,4744553,telit001,MIA,20161002WAS0,D2,Catcher,defense
7,5589581,bourj002,MIA,20161002WAS0,D3,1st Base,defense
8,4462877,gordd002,MIA,20161002WAS0,D4,2nd Base,defense
9,5026229,pradm001,MIA,20161002WAS0,D5,3rd Base,defense


In [ ]:
tables = [
    "game_log",
    "park_codes",
    "team_codes",
    "person_codes"
]

for t in tables:
    c = '''
    DROP TABLE {}
    '''.format(t)
    
    run_command(c)

show_tables()

In [17]:
show_tables()

,name,type,sql
0,park,table,CREATE TABLE park\n (\n park_id TEXT PRI...
1,league,table,CREATE TABLE league\n (\n league_id TEXT...
2,team,table,CREATE TABLE team (\n team_id TEXT PRIMARY ...
3,game,table,CREATE TABLE game (\n game_id TEXT PRIMARY ...
4,team_appearance,table,CREATE TABLE team_appearance (\n team_id TE...
5,person,table,CREATE TABLE person\n (\n person_id TEXT...
6,appearance_type,table,CREATE TABLE appearance_type (\n appearance...
7,game_log,table,"CREATE TABLE ""game_log"" (\n""date"" INTEGER,\n ..."
8,person_codes,table,"CREATE TABLE ""person_codes"" (\n""id"" TEXT,\n ""..."
9,team_codes,table,"CREATE TABLE ""team_codes"" (\n""team_id"" TEXT,\n..."


In [ ]:
c = 'drop table appearance_type'
run_command(c)

In [31]:
c = 'DROP TABLE test'
c1 = '''
    CREATE TABLE  test
    (
        text_1 TEXT,
        text_2 TEXT
    )

'''
run_command(c)
run_command(c1)

c2 = 'DROP TABLE TESTED'
c3 = '''
   CREATE TABLE tested
   (
       num  INT,
       integer INT
   )
'''
run_command(c2)
run_command(c3)

In [42]:
c = '''
    INSERT INTO tested
    (
        num,
        integer
    )
    VALUES
    (8,9)
'''

run_command(c)


In [53]:
q = '''
    SELECT count(*)
    FROM game_log
    
'''
run_query(q)

,count(*)
0,171907
